In [5]:
import pandas as pd
pd.set_option("display.max_colwidth", 1)
import py2neo

graph = py2neo.Graph()
graph

<Graph uri='http://localhost:7474/db/data/'>

In [31]:
import pandas as pd

query="""
MATCH (p:Project)-[:CONTAINS*]->(t:Type)
WITH t, SPLIT(t.fqn, ".")[-2] as buildingblock
MERGE (b:Abb { name : buildingblock})
MERGE (b)-[:CONSISTS_OF]->(t)
RETURN b.name, t.name
"""
pd.DataFrame(graph.data(query)).head()

,b.name,t.name
0,jdbc,JdbcVisitRepositoryImpl
1,springdatajpa,SpringDataPetRepository
2,model,Visit
3,jdbc,JdbcPet
4,model,Specialty


In [35]:
query="""
MATCH
    (s1:Subdomain)<-[:BELONGS_TO]-
    (type:Type)-[:DECLARES]->(m:Method)-[r:INVOKES]->(mdep:Method)<-[:DECLARES]-
    (dependency:Type)-[:BELONGS_TO]->(s2:Subdomain)
RETURN s1.name as from, s2.name as to, COUNT(r) as x_number
"""
df = pd.DataFrame(graph.data(query))
df.head()

,from,to,x_number
0,Clinic,Visit,8
1,Clinic,Clinic,28
2,Visit,Visit,15
3,Clinic,Vet,5
4,Clinic,Specialty,2


In [36]:
import json
json_data = df.to_dict(orient='split')['data']
with open ( "vis/chord_data.json", mode='w') as json_file:
    json_file.write(json.dumps(json_data, indent=3))
json_data[:5]

[['Clinic', 'Visit', 8],
 ['Clinic', 'Clinic', 28],
 ['Visit', 'Visit', 15],
 ['Clinic', 'Vet', 5],
 ['Clinic', 'Specialty', 2]]

In [39]:
query="""
MATCH
    (s1:Subdomain { name: "Owner"})<-[:BELONGS_TO]-
    (type:Type)-[:DECLARES]->(m:Method)-[r:INVOKES]->(mdep:Method)<-[:DECLARES]-
    (dependency:Type)-[:BELONGS_TO]->(s2:Subdomain {name: "Clinic"})
RETURN type.name as from, dependency.name as to, m.name, r.lineNumber, mdep.name
"""
df = pd.DataFrame(graph.data(query))
df

,from,m.name,mdep.name,r.lineNumber,to
0,OwnerControllerTests,testProcessFindFormByLastName,findOwnerByLastName,113,ClinicService
1,OwnerControllerTests,testProcessFindFormSuccess,findOwnerByLastName,104,ClinicService
2,OwnerControllerTests,setup,findOwnerById,56,ClinicService
3,OwnerController,showOwner,findOwnerById,134,ClinicService
4,OwnerController,processUpdateOwnerForm,saveOwner,120,ClinicService
5,OwnerController,initUpdateOwnerForm,findOwnerById,109,ClinicService
6,OwnerController,processFindForm,findOwnerByLastName,91,ClinicService
7,OwnerController,processCreationForm,saveOwner,71,ClinicService
